# SVG Furniture Catalog Extraction

**Goal**: Parse SVG file to extract furniture dimensions

## Steps:
1. Load SVG file with furniture definitions
2. Extract `<path class="bg">` for each furniture ID
3. Calculate bbox → width/height
4. Save as `furniture_catalog.json`

In [ ]:
import sys
sys.path.append('../utils')

from svg_parser import (
    extract_furniture_from_svg,
    parse_svg_path_bbox,
    bbox_to_dimensions,
    save_catalog_json
)
import json
from pathlib import Path

## 1. Test Path Parser

Test with your example path

In [ ]:
# Example from your SVG
test_path = "M193 0A7 7 0 0 0 200 -7v-86A7 7 0 0 0 193 -100h-186A7 7 0 0 0 0 -93v86A7 7 0 0 0 7 0z"

bbox = parse_svg_path_bbox(test_path)
print(f"Bounding box: {bbox}")
print(f"  min_x={bbox[0]}, min_y={bbox[1]}, max_x={bbox[2]}, max_y={bbox[3]}")

dims = bbox_to_dimensions(bbox, scale_factor=1.0)  # 1px = 1cm
print(f"\nDimensions (assuming 1px=1cm):")
print(f"  Width: {dims['width']:.2f}m")
print(f"  Height: {dims['height']:.2f}m")

# If scale is different, adjust:
# dims = bbox_to_dimensions(bbox, scale_factor=YOUR_SCALE)

## 2. Extract from Full SVG Catalog

**TODO**: Update `svg_path` to your furniture catalog file

In [ ]:
# TODO: Update this path!
svg_path = "../samples/furniture_catalog.svg"  # Or wherever you'll put it

# Check if file exists
if Path(svg_path).exists():
    catalog = extract_furniture_from_svg(svg_path)
    print(f"✅ Extracted {len(catalog)} furniture items\n")
    
    # Show first 10
    for furn_id, data in list(catalog.items())[:10]:
        print(f"{furn_id:15s}: {data['width']:.2f}m x {data['height']:.2f}m")
else:
    print(f"⚠️  File not found: {svg_path}")
    print("Upload your furniture catalog SVG to samples/ folder tomorrow")

## 3. Inspect Sample Furniture Items

In [ ]:
# If catalog was extracted, inspect specific items
if 'catalog' in locals():
    # Check if expected IDs are present (from your JSON example)
    expected_ids = ['b2', 'sz', 'sh', 'wc', 'kitchenette']
    
    print("Looking for expected furniture IDs:\n")
    for furn_id in expected_ids:
        if furn_id in catalog:
            data = catalog[furn_id]
            print(f"✅ {furn_id}:")
            print(f"   Width: {data['width']:.2f}m")
            print(f"   Height: {data['height']:.2f}m")
            print(f"   BBox: {data['bbox']}")
        else:
            print(f"❌ {furn_id}: NOT FOUND")
        print()

## 4. Verify Scale Factor

Check if dimensions make sense (e.g., double bed ~2m x 1.5m)

In [ ]:
# Common furniture expected sizes (for validation)
expected_sizes = {
    'b2': {'width': (1.8, 2.2), 'height': (1.4, 1.8)},  # Double bed
    'sz': {'width': (0.4, 0.8), 'height': (0.4, 0.8)},  # Nightstand
    'sh': {'width': (0.8, 1.2), 'height': (0.8, 1.2)},  # Shower
    'wc': {'width': (0.4, 0.7), 'height': (0.5, 0.8)},  # Toilet
}

if 'catalog' in locals():
    print("Validating dimensions against expected ranges:\n")
    for furn_id, expected in expected_sizes.items():
        if furn_id in catalog:
            actual = catalog[furn_id]
            w_ok = expected['width'][0] <= actual['width'] <= expected['width'][1]
            h_ok = expected['height'][0] <= actual['height'] <= expected['height'][1]
            
            status = "✅" if (w_ok and h_ok) else "⚠️ "
            print(f"{status} {furn_id}: {actual['width']:.2f}m x {actual['height']:.2f}m")
            
            if not (w_ok and h_ok):
                print(f"   Expected: {expected['width'][0]}-{expected['width'][1]}m x {expected['height'][0]}-{expected['height'][1]}m")
                print(f"   → Check SVG_SCALE_FACTOR in svg_parser.py")

## 5. Save Catalog to JSON

In [ ]:
if 'catalog' in locals() and catalog:
    output_path = "../samples/furniture_catalog.json"
    save_catalog_json(catalog, output_path)
    
    # Also show summary stats
    widths = [d['width'] for d in catalog.values()]
    heights = [d['height'] for d in catalog.values()]
    
    print(f"\nCatalog statistics:")
    print(f"  Total items: {len(catalog)}")
    print(f"  Width range: {min(widths):.2f}m - {max(widths):.2f}m")
    print(f"  Height range: {min(heights):.2f}m - {max(heights):.2f}m")
else:
    print("⚠️  No catalog to save. Upload SVG file first.")

## Next Steps

Tomorrow:
1. ✅ Upload furniture catalog SVG to `samples/`
2. ✅ Upload sample apartment JSONs to `samples/`
3. 🔄 Run this notebook to extract catalog
4. 🔄 Run `eda_apartments.ipynb` to analyze apartments
5. 🔄 Start building data pipeline